# Delete this as soon as you see it and don't know why its here.

In [1]:
import json

with open("representative_questions.json", "r", encoding="utf-8") as f:
    representative_questions = json.load(f)


with open("edital_name_to_file_path.json", "r",  encoding="utf-8") as f:
    edital_name_to_file_path = json.load(f)["editais"]

In [2]:


from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_to_edital = []
for edital in representative_questions["editais"].keys():
    for question in representative_questions["editais"][edital]:
        vector = embeddings.embed_query(question)
        vector_to_edital.append((vector, edital))

C:\Users\Miguel_Rodrigues\AppData\Local\Temp\ipykernel_20020\1842490047.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\Miguel_Rodrigues\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from scipy.spatial.distance import cosine

def find_edital_by_question(question):
    question_vector = embeddings.embed_query(question)
    closest_edital = None
    similarities = []
    # biggest_similarity = -1  # cosine similarity ranges [-1, 1]

    for vector, edital in vector_to_edital:
        cos_sim = 1 - cosine(question_vector, vector)
        # if cos_sim > biggest_similarity:
        #     biggest_similarity = cos_sim
        #     closest_edital = edital
        similarities.append((cos_sim, edital))
    
    similarities.sort(reverse=True)
    editais_count = {}
    for similiraty in similarities[0:5]:
        edital = similiraty[1]
        if edital in editais_count.keys():
            editais_count[edital] += 1
        else:
            editais_count[edital] = 1

    print(similarities[0:5])

    if similarities[0][0] > 0.9:
        return similarities[0][1]

    max_count = -1
    max_edital = None
    for edital, value in editais_count.items():
        if value > max_count:
            max_count = value
            max_edital = edital

    return max_edital

def get_edital_file_content(edital_name):
    """
    Given the name of an edital, returns its content as a string.
    """
    file_path = edital_name_to_file_path[edital_name]
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()
    

find_edital_by_question("Qual o objetivo do curso de Ciência da Computação da UFCG?")


[(np.float64(1.0), 'PPC Ciência da Computação (Bacharelado) – UFCG'), (np.float64(0.7183902126156625), 'Edital Nº 01/ETSC Retificado/2025'), (np.float64(0.669871863366076), 'AUXÍLIO AATIVIDADES OBRIGATÓRIAS EXTERNAS'), (np.float64(0.6672988348418725), 'Edital Nº 02/2025 UFCG (Procedimento Complementar à Autodeclaração PPI/PCD)'), (np.float64(0.6599877479428219), 'AUXÍLIO AATIVIDADES OBRIGATÓRIAS EXTERNAS')]


'PPC Ciência da Computação (Bacharelado) – UFCG'

In [5]:
import importlib
import prompt
import call_model_dell

# Reload modules
importlib.reload(prompt)
importlib.reload(call_model_dell)

# Re-import functions to get the latest definitions
from prompt import prompt
from call_model_dell import call_model

def answer_question(question):
    edital_name = find_edital_by_question(question)
    edital_content = get_edital_file_content(edital_name)

    full_prompt = prompt(edital_content, question)

    answer = call_model(full_prompt)

    return answer, edital_name

x = answer_question("Quais requisitos devo atender para participar do cadastro socioeconômico?")

def print_x(x):
    def print_with_line_breaks(text, n=10):
        words = text.split()
        for i in range(0, len(words), n):
            print(' '.join(words[i:i+n]))

    print("Edital:", x[1])
    print_with_line_breaks(x[0], 10)

print_x(x)

[(np.float64(0.9147705874232672), 'Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)'), (np.float64(0.8315611680201419), 'Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)'), (np.float64(0.8128246657534475), 'Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)'), (np.float64(0.7995349516003089), 'Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)'), (np.float64(0.7977582746090693), 'Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)')]
Edital: Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)
Resposta = Para participar do processo de cadastramento socioeconômico, o/a
estudante deve atender cumulativamente aos seguintes requisitos: a. Estar regularmente
matriculado o u cadastrado para ingresso em cursos presenciais de
graduação da UFCG. b. Possuir grupo familiar com renda per
capita(por pessoa) menor ou igual a 1 (um) salário- mínimo,
entendendo-se por renda mensal per capita o valor bruto dos
rendimentos da família, dividido pelo número

In [10]:
x = answer_question("Qual é a carga horária do curso de Computação da UFCG?")

print_x(x)

[(np.float64(0.8272548646358513), 'PPC Ciência da Computação (Bacharelado) – UFCG'), (np.float64(0.6814681304119237), 'Edital Nº 01/ETSC Retificado/2025'), (np.float64(0.6589174100425531), 'Edital Nº 02/2025 UFCG (Procedimento Complementar à Autodeclaração PPI/PCD)'), (np.float64(0.654725602816948), 'PPC Ciência da Computação (Bacharelado) – UFCG'), (np.float64(0.6511330053539607), 'AUXÍLIO AATIVIDADES OBRIGATÓRIAS EXTERNAS')]
Edital: PPC Ciência da Computação (Bacharelado) – UFCG
Resposta = O curso de Ciência da Computação tem uma
duração mínima de 3.270 (três mil, duzentas e setenta) horas,
correspondendo a 218 (duzentos e dezoito) créditos. Essa resposta está
no contexto. A resposta final é: O curso de Ciência
da Computação tem uma duração mínima de 3.270 (três mil,
duzentas e setenta) horas, correspondendo a 218 (duzentos e dezoito)
créditos.


In [23]:
x

(' \nResposta = Não consigo responder a essa pergunta com minha base de informações.',
 'AUXÍLIO AATIVIDADES OBRIGATÓRIAS EXTERNAS')